# 🛒 Grocery Store – Training an Image Classifier

In this notebook, we’ll build and train a simple image classification model to recognize grocery items from photos. The dataset contains labeled images of various food products, each assigned to a coarse category like "vegetables", "dairy", or "snacks".

We’ll walk through a full training workflow, including:

- Loading and preprocessing image data
- Defining a PyTorch dataset and data loaders
- Fine-tuning a pretrained MobileNet model
- Evaluating the model's performance on a validation set
- Visualizing sample predictions with images

By the end, you'll have a basic image classifier running locally, ready to be served or deployed in later steps of the workshop.

**📚 Importing Packages**

This section imports all necessary Python libraries:

- `os`, `pandas`, `matplotlib.pyplot`: for file handling, tabular data, and visualization
- `torch`, `torchvision`, `torch.nn`, `torch.optim`: core PyTorch libraries for deep learning
- `PIL.Image`: for loading and manipulating image files
- `tqdm`: to show progress bars during training loops

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

**⚙️ Configuration and Class Labels**

This section defines basic training parameters and paths to the dataset files.

- Sets the **data directory** and **file locations** for class labels, training set, and validation set.
- Defines training **hyperparameters** like batch size, learning rate, and number of epochs.
- Detects whether a **GPU is available** and selects the appropriate device (CPU or CUDA).
- Loads the mapping from numeric **class IDs** to human-readable names from `classes.csv`.

**NOTE**: This notebook relies on `~/GroceryStoreDataset` already being present in the home folder.

If it is not, simply run: `git clone https://github.com/marcusklasson/GroceryStoreDataset`

In [ ]:
# Config
DATA_DIR = os.path.expanduser('~/GroceryStoreDataset/dataset')
CLASSES_CSV = os.path.join(DATA_DIR, 'classes.csv')
TRAIN_CSV = os.path.join(DATA_DIR, 'train.txt')
VAL_CSV = os.path.join(DATA_DIR, 'val.txt')
BATCH_SIZE = 32
NUM_EPOCHS = 5
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load class names
CLASSES = (  # dict(int => class name)
    pd.read_csv(CLASSES_CSV)
    .drop_duplicates("Coarse Class ID (int)")
    .set_index("Coarse Class ID (int)")["Coarse Class Name (str)"]
    .to_dict()
)

**🧺 Dataset, Data Loaders, and Model Setup**

This section prepares the **dataset** and sets up the **model**. We define a custom `GroceryDataset` class that loads images and labels from CSV files provided in the Grocery Store Dataset. Each image has both a fine-grained label (specific product) and a coarse-grained label (product category). In this notebook, we focus on the **coarse labels** for simplicity.

To prepare the images, we apply a basic **preprocessing** pipeline: **resizing** each image to 224×224 pixels, converting it to a PyTorch tensor, and **normalizing** the pixel values using the standard mean and standard deviation from ImageNet. This normalization ensures the input distribution matches what the MobileNet model expects.

We use PyTorch `DataLoader`s to handle batching and shuffling of the training and validation data. For the model, we load a pre-trained **MobileNetV3 Small** architecture — a compact and efficient network well-suited for edge deployment. We replace the final **classification layer** to match the number of classes in our dataset and move the model to the appropriate device (CPU or GPU).


In [ ]:
# Dataset class
class GroceryDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file, header=None)
        self.df.columns = ["path", "fine_label", "coarse_label"]
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(DATA_DIR, row["path"])
        label = row["coarse_label"]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Data loaders
train_dataset = GroceryDataset(TRAIN_CSV, transform=transform)
val_dataset = GroceryDataset(VAL_CSV, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model
model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(CLASSES))
model = model.to(DEVICE)

**🏋️ Training the Model**

We now set up the **training loop**. The model is trained using the **Adam optimizer** and **cross-entropy loss**, which is suitable for multi-class classification tasks.

For each epoch, we iterate over the training dataset in batches. We move data to the correct device, perform a **forward pass** through the model, **compute the loss**, **backpropagates** the gradients, and **update the model** weights. The average training loss per epoch is printed to track progress over time.

In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
    print(f"Epoch {epoch+1} - Loss: {total_loss / len(train_loader.dataset):.4f}")

**📊 Model Evaluation**

After training, we evaluate the model on the **validation set** to check **how well it generalizes**. We switch the model to **evaluation mode** and disable gradient tracking for efficiency. For each batch, we compute predictions and compare them to the ground truth labels. At the end, we print the overall **validation accuracy** as a percentage.

In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"\nValidation Accuracy: {correct / total:.2%}")

**🖼️ Visualizing Predictions**

To better understand the model's behavior, we display **10 sample predictions** from the validation set. For each image, we show the predicted and true class names. The pixel values are unnormalized for display so the images look natural. This kind of qualitative inspection is useful for spotting obvious misclassifications or biases in the model.

In [ ]:
# Show predictions with images
print("\nSample predictions with images:")
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

# Get 10 images, predictions, and labels again for visualization
shown = 0
for images, labels in val_loader:
    images_cpu = images.cpu()
    outputs = model(images.to(DEVICE))
    _, predicted = torch.max(outputs, 1)
    predicted = predicted.cpu()

    for i in range(images.size(0)):
        if shown >= 10:
            break
        img = images_cpu[i].permute(1, 2, 0).numpy()
        img = img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]  # unnormalize
        img = img.clip(0, 1)

        axes[shown].imshow(img)
        axes[shown].axis("off")
        axes[shown].set_title(f"Predict: {CLASSES[predicted[i].item()]}\nTrue label: {CLASSES[labels[i].item()]}")
        shown += 1

    if shown >= 10:
        break

plt.tight_layout()
plt.show()